<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/intervviewZero_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install m3u8

In [ ]:
!pip install git+https://github.com/openai/whisper.git
#!pip install -U git+https://github.com/jianfch/stable-ts.git

In [ ]:
!pip install moviepy

In [ ]:
!pip install ffmpeg-python

In [ ]:
!pip install pydub

In [ ]:
import whisper
#import stable_whisper
import ffmpeg
from pydub import AudioSegment
import numpy as np
import json, re, os
from datetime import datetime
import glob
import locale
locale.getpreferredencoding = lambda: "UTF-8"
from moviepy.editor import VideoFileClip, CompositeVideoClip
from moviepy.editor import AudioFileClip, CompositeAudioClip, concatenate_audioclips
from moviepy.audio.AudioClip import AudioArrayClip
import m3u8

In [7]:
!cp ./drive/MyDrive/IZ-2218確認５/*.* ./
#!cp ./drive/MyDrive/IZ-2218確認４/*.* ./
#!cp ./drive/MyDrive/IZ-2218確認３/*.* ./

In [8]:
def is_webm(path):
    with open('./' + path) as f:
        for _line in f:
            if _line.rstrip()[-5:] == '.webm':
                return True
    return False

In [9]:
def conv_webm(input_file):
    #!ffmpeg -allowed_extensions ALL -f hls -i ./drive/MyDrive/IZ-2218確認３/ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.m3u8 -c:v copy -y ./ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm
    webm_tmp = '.'.join(input_file.split('.')[:-1]) + '.webm'
    try:
        (
            ffmpeg
            .input('./' + input_file, allowed_extensions='ALL')
            .output(webm_tmp, vcodec="copy")
            .overwrite_output()
            .run(capture_stderr=True, capture_stdout=True)
        )
        _clip = VideoFileClip(webm_tmp)
    except ffmpeg.Error as e:
        print(f"{e.stderr.decode()}")
        print(f"標準出力: {e.stdout.decode()}")
        _clip = None
    return _clip, webm_tmp

In [10]:
def get_video_list(path):
    video_path = f'{path}*_video.m3u8'
    video_paths = glob.glob(video_path)
    video_paths.sort()
    return video_paths
def get_audio_list(path):
    audio_path = f'{path}*_audio.m3u8'
    audio_paths = glob.glob(audio_path)
    audio_paths.sort()
    return audio_paths

In [11]:
def conv_unix_time(date_string):
    year = int(date_string[:4])
    month = int(date_string[4:6])
    day = int(date_string[6:8])
    hour = int(date_string[8:10])
    minute = int(date_string[10:12])
    second = int(date_string[12:14])
    millisecond = int(date_string[14:])
    dt = datetime(year, month, day, hour, minute, second)
    unix_time = int(dt.timestamp()) * 1000 + millisecond
    return unix_time

In [12]:
def add_silent_segment(input_audio_path, output_audio_path, check=False):
    m3u8_obj = m3u8.load(input_audio_path)
    segments = m3u8_obj.segments

    audio_clips = []
    previous_end_time = 0  # 無音の計算用
    duration = 0

    for segment in segments:
        segment_url = segment.uri
        segment_duration = segment.duration
        segment_start_time = conv_unix_time(segment.uri.split('_')[-1].split('.')[0])

        audio = AudioFileClip(segment_url)

        # 無音が必要なら追加
        if  segment.media_sequence > 0 and previous_end_time < segment_start_time:
            #print(segment.media_sequence, '無音', segment_start_time, ' > ', previous_end_time, (segment_start_time - previous_end_time) / 1000, flush=True)
            silence_duration = (segment_start_time - previous_end_time) / 1000
            silence = AudioArrayClip(np.zeros((int(silence_duration * audio.fps), 2)), fps=audio.fps)
            audio_clips.append(silence)
            duration += silence_duration
            if check == True:
                _check = True
                break

        if check == True:
            if _check == True:
                return True
            else:
                return False

        previous_end_time = segment_start_time + int(segment_duration * 1000)

        # 音声をリストに追加
        audio_clips.append(audio)

    # すべての音声を結合
    final_audio = concatenate_audioclips(audio_clips)
    return final_audio


In [13]:
def get_speaker_monitor(result, sorted, video_paths):
    prog = re.compile('教えて|聞か|聞き|願い|ください|ですか(?!ね)|でしょうか(?!ね)|ましたか|されましたか|いつ頃')
    character = {}
    INT_COUNT = []
    for c in range(len(result)):
        INT_COUNT.append([0, 0, 0, ''])
    for srt, end, txt, spk in sorted:
        count = len(prog.findall(txt))
        spk_no = int(spk.split('_')[-1])
        INT_COUNT[spk_no][0] += 1
        INT_COUNT[spk_no][3] = spk
        if count > 0:
            INT_COUNT[spk_no][1] += count
    for c, cnt in enumerate(INT_COUNT):
        INT_COUNT[c][2] = cnt[1] / cnt[0]
    INT_COUNT.index(min(INT_COUNT))

    rate = [c[2] for c in INT_COUNT]
    min_index = rate.index(min(rate))

    for i, intv in enumerate(INT_COUNT):
        user_id = audio_paths[i].split('_')[-5]
        if i == min_index:
            character[intv[3]] = {'character': "モニター", 'user_id': user_id, 'video_path' : video_paths[i]}
        else:
            character[intv[3]] = {'character': "インタビューア_" + intv[3].split('_')[-1], 'user_id': user_id, 'video_path' : video_paths[i]}
    return character

In [14]:
path = './'
video_paths = get_video_list(path)
audio_paths = get_audio_list(path)
interview_video_name = '_'.join(video_paths[0].split('/')[1].split('_')[0: 2]) + '.mp4'
interview_audio_name = '_'.join(video_paths[0].split('/')[1].split('_')[0: 2]) + '.wav'
interview_json_name = '_'.join(video_paths[0].split('/')[1].split('_')[0: 2]) + '.json'

In [ ]:
video = []
for v, video_path in enumerate(video_paths):
    _video, _path = conv_webm(video_path) if is_webm(video_path) else (VideoFileClip(video_path), None)
    if _path is not None:
        video_paths[v] = _path
    video.append(_video)
video_paths

In [ ]:
audio = []
wav_paths = []
for audio_path in audio_paths:
    wav_path = '.'.join(audio_path.split('.')[:-1]) + '.wav'
    if add_silent_segment(input_audio_path, output_audio_path, check=True):
        audio_tmp = add_silent_segment(audio_path, wav_path)
        print("無音あり")
    else:
        audio_tmp = AudioFileClip(audio_path)
        print("無音なし")
    audio_tmp.write_audiofile(wav_path, ffmpeg_params=["-ac", "1"])
    print(f'duration : {audio_tmp.duration}')
    audio.append(audio_tmp)
    wav_paths.append(wav_path)
print(audio)

In [ ]:
model = whisper.load_model('large')
#model = stable_whisper.load_model('large')
#stable_whisper.modify_model(model)

In [ ]:
asr_result = []
for wav_patn in wav_paths:
    print(f'\n---- {wav_patn} ----')
    #_result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True)
    _result = model.transcribe(wav_patn, verbose=True, fp16=False, language="ja", word_timestamps=True, logprob_threshold=0, no_speech_threshold=0.4, compression_ratio_threshold=2.6)
    asr_result.append(_result)

In [19]:
_seg = []
for spk, res in enumerate(asr_result):
    for seg in res['segments']:
        _seg.append([seg['start'], seg['end'], seg['text'], f'speaker_{format(spk, "02") }'])
#_seg = []
#for spk, res in enumerate(asr_result):
#    for seg in res.segments:
#        _seg.append([seg.start, seg.end, seg.text, f'speaker_{format(spk, "02") }'])

In [20]:
segments_sorted = sorted(_seg, key=lambda x:(x[0], x[1]))

In [ ]:
characters = get_speaker_monitor(asr_result, segments_sorted, video_paths)
print(characters)

In [22]:
InterReview_json = {"movie_name": interview_video_name}
speak_json = []
for start, end, text, speaker in segments_sorted:
    _speak = {"start": f'{start:.2f}s', "end": f'{end:.2f}s', 'speaker': characters[speaker]['character'], 'speak': text}
    speak_json.append(_speak)
InterReview_json['InterReview'] = speak_json

with open('./' + interview_json_name, 'w', encoding='utf-8') as f:
    json.dump(InterReview_json, f, indent=4, ensure_ascii=False)

In [23]:
video_part = ['']  * len(characters)
cnt = 1
for c, char in enumerate(characters):
    if characters[char]['character'] == 'モニター':
        #video_part[0] = video[c]
        video_part[0] = characters[char]['video_path']
    else:
        #video_part[cnt] = video[c]
        video_part[cnt] = characters[char]['video_path']
        cnt += 1

In [24]:
#w,h = moviesize = video_part[0].size
main_w, main_h = moviesize = VideoFileClip(video_part[0]).size
wipe_w = 128
wipe_h = 72
wipe_margin_right = 4
wipe_margin_top = 4

In [25]:
#################### moviepy は出力が遅いので ffmpeg-python を利用

#comp_mp4 = [video_part[0]]
#for v in range(len(video_part) - 1):
#    _wipe = (video_part[v + 1].
#            resize((wipe_w, wipe_h)).
#            margin(top=wipe_margin_top, right=wipe_margin_right, opacity=0).
#            set_pos((main_w - wipe_w - wipe_margin_right, wipe_h * 0 + wipe_margin_right * 1)) )
#    comp_mp4.append(_wipe)

In [26]:
#################### moviepy は出力が遅いので ffmpeg-python を利用
#audio_clip = AudioFileClip(interview_audio_name)

#combined_clip = CompositeVideoClip(comp_mp4)
#combined_clip = combined_clip.set_audio(audio_clip)

#wav_clips = []
#for w_path in wav_paths:
#    print(w_path)
#    wav_clip = AudioFileClip(w_path)
#    wav_clip.set_fps(44100)
#    wav_clips.append(wav_clip)
#composite_audio = CompositeAudioClip(wav_clips)
#composite_audio.write_audiofile('./' + interview_audio_name, ffmpeg_params=["-ac", "1"])

In [27]:
#################### moviepy は出力が遅いので ffmpeg-python を利用

#combined_clip.write_videofile(interview_vido_name, fps=30, audio_codec = 'aac')

In [ ]:
comp_wav = AudioSegment.from_file(wav_paths[0])
for w in range(len(wav_paths) - 1):
    orver_wav = AudioSegment.from_file(wav_paths[w + 1])
    comp_wav = comp_wav.overlay(orver_wav, position=0)
comp_wav.export(interview_audio_name, format="wav")

In [29]:
voice = ffmpeg.input(interview_audio_name)

video_clip = []
for part in video_part:
    _clop = ffmpeg.input(part)
    video_clip.append(_clop)

sub_scaled = []
for c, clip in enumerate(video_clip):
    if c == 0: continue
    _scaled = clip.filter('scale', wipe_w, wipe_h)
    sub_scaled.append(_scaled)

orverlay = []
for s, scl in enumerate(sub_scaled):
    if s == 0:
        sub1 = video_clip[0]
        sub1 = sub1.filter('fps', fps=24, round='up')
    else:
        sub1 = orverlay[s - 1]
    sub2 = scl
    _orverlay = ffmpeg.overlay(sub1, sub2, x=(main_w - wipe_w - wipe_margin_right), y=(wipe_h * s + wipe_margin_top * (s + 1)) )
    orverlay.append(_orverlay)

output = ffmpeg.output(
    orverlay[len(orverlay) - 1], voice.audio,
    interview_video_name,
    vcodec='libx264', acodec='aac', audio_bitrate='192k', preset='fast'
)

output = output.overwrite_output()


In [30]:
try:
    ffmpeg.run(output, capture_stderr=True, capture_stdout=True)
except ffmpeg.Error as e:
    print(f"標準エラー{e.stderr.decode()}")
    print(f"標準出力: {e.stdout.decode()}")

In [31]:
'''
# 入力映像と音声ファイルの設定
main = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm')
sub1 = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_462504175__uid_e_video.m3u8')
sub2 = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_369679075__uid_e_video.m3u8')
audio = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981.wav')

# サブ映像のリサイズ
sub1_scaled = sub1.filter('scale', wipe_w, wipe_h)
sub2_scaled = sub2.filter('scale', wipe_w, wipe_h)

# サブ映像1を右下にオーバーレイ
overlay1 = ffmpeg.overlay(main, sub1_scaled, x=(main_w - wipe_w - wipe_margin_right), y=(wipe_h * 0 + wipe_margin_top * (0 + 1)))

# サブ映像2を左上にオーバーレイ
overlay2 = ffmpeg.overlay(overlay1, sub2_scaled, x=(main_w - wipe_w - wipe_margin_right), y=(wipe_h * s + wipe_margin_top * (1 + 1)))

# 映像ストリームを取得
#video_stream = overlay2.video

# メイン映像の音声ストリームを取得
#main_audio = main.audio

# 音声のミックス（メイン映像の音声と外部音声をミックス）
#mixed_audio = ffmpeg.filter([main_audio, audio], 'amix', inputs=2, duration='longest')

# 映像と音声を結合して出力
output = ffmpeg.output(
    overlay2, audio.audio,
    interview_video_name,
    vcodec='libx264', acodec='aac', audio_bitrate='192k', preset='fast'
)
output = output.overwrite_output()

# 実行
ffmpeg.run(output)
'''

"\n# 入力映像と音声ファイルの設定\nmain = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_907719631__uid_e_video.webm')\nsub1 = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_462504175__uid_e_video.m3u8')\nsub2 = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981__uid_s_369679075__uid_e_video.m3u8')\naudio = ffmpeg.input('ee93aa2994411ebff28d459c4711228b_1731555758981.wav')\n\n# サブ映像のリサイズ\nsub1_scaled = sub1.filter('scale', wipe_w, wipe_h)\nsub2_scaled = sub2.filter('scale', wipe_w, wipe_h)\n\n# サブ映像1を右下にオーバーレイ\noverlay1 = ffmpeg.overlay(main, sub1_scaled, x=(main_w - wipe_w - wipe_margin_right), y=(wipe_h * 0 + wipe_margin_top * (0 + 1)))\n\n# サブ映像2を左上にオーバーレイ\noverlay2 = ffmpeg.overlay(overlay1, sub2_scaled, x=(main_w - wipe_w - wipe_margin_right), y=(wipe_h * s + wipe_margin_top * (1 + 1)))\n\n# 映像ストリームを取得\n#video_stream = overlay2.video\n\n# メイン映像の音声ストリームを取得\n#main_audio = main.audio\n\n# 音声のミックス（メイン映像の音声と外部音声をミックス）\n#mixed_audio = ffmpeg.filter(